In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
from spacy import displacy
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from hp_nlp_graph.scraper import Chapter, Character
from hp_nlp_graph.coreference import (
    coref_resolve_and_get_characters_matches_in_chapter,
    get_interactions,
)
from hp_nlp_graph.language import (
    get_coref_resolver_nlp,
    add_entity_ruler,
    get_matcher,
    FastCoref,
)
from hp_nlp_graph.utils import get_characters_seen_till_chapter
from hp_nlp_graph.language_constants import CHAPTER_HARDCODED_OPTIONS

In [2]:
book_number = 7
book_text_path = "./data/books/7 Deathly Hallows.txt"

In [3]:
chapter_characters = {}
for book_number_ in range(1, 8):
    chapter_characters[book_number_] = pickle.load(
        open(f"data/processed/{book_number_}/chapter_characters.pkl", "rb")
    )
characters = [
    character
    for book_number_, chapters in chapter_characters.items()
    for chapter in chapters
    for character in chapter.characters
]
chapter_characters = {
    book_number_: {chapter.chapter: chapter.characters for chapter in chapters}
    for book_number_, chapters in chapter_characters.items()
}
chapters = open(book_text_path, "r").read().split("CHAPTER ")[1:]

In [4]:
base_nlp, nlp = get_coref_resolver_nlp(device="cuda:0")
base_nlp = add_entity_ruler(base_nlp, characters)
nlp = add_entity_ruler(nlp, characters)
coref = FastCoref(base_nlp, nlp)

11/01/2023 14:52:08 - INFO - 	 missing_keys: []
11/01/2023 14:52:08 - INFO - 	 unexpected_keys: []
11/01/2023 14:52:08 - INFO - 	 mismatched_keys: []
11/01/2023 14:52:08 - INFO - 	 error_msgs: []
11/01/2023 14:52:08 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M


In [5]:
# from hp_nlp_graph.language import REMOVE_WORDS

# matcher = get_matcher(base_nlp, get_character_dict(chapter_characters, 7, 37))
# s = "Hogwarts house-elves"
# print(matcher.has_key(s), matcher.get(s))
# print(set(s.split(" ")).isdisjoint(set(REMOVE_WORDS)))

In [6]:
interactions_by_chapter = {}
# with open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "rb") as f:
#     interactions_by_chapter = pickle.load(f)
for chapter in range(1, len(chapters) + 1):
    # for chapter in range(37, len(chapters) + 1):
    result, resolved_doc = coref_resolve_and_get_characters_matches_in_chapter(
        base_nlp=base_nlp,
        nlp=nlp,
        chapter_text=chapters[chapter - 1],
        characters_seen_till_this_chapter=get_characters_seen_till_chapter(
            chapter_characters, book_number, chapter
        ),
        coref_resolver=coref.resolve,
        chapter_hardcoded_options=CHAPTER_HARDCODED_OPTIONS[f"{book_number}-{chapter}"]
        if f"{book_number}-{chapter}" in CHAPTER_HARDCODED_OPTIONS
        else None,
    )
    interactions_by_chapter[chapter] = dict(get_interactions(result, 14))

The Dark Lord Ascending


11/01/2023 14:52:09 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/01/2023 14:52:10 - INFO - 	 ***** Running Inference on 1 texts *****
11/01/2023 14:52:12 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

11/01/2023 14:55:36 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/01/2023 14:55:36 - INFO - 	 ***** Running Inference on 1 texts *****
11/01/2023 14:55:37 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/01/2023 14:55:37 - INFO - 	 ***** Running Inference on 1 texts *****


In [8]:
with open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "wb") as f:
    pickle.dump(interactions_by_chapter, f)